In [2]:
pkgs = [
    "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121"
]

for p in pkgs:
    print(f"\nInstalling {p} ...")
    !{sys.executable} -m pip install {p}


Installing torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 ...


'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import re

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [31]:
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
def categorize_template(template):
    """
    Categorize interaction templates into high-level classes
    """
    template_lower = template.lower()
    
    if 'adverse effects' in template_lower and 'risk or severity' in template_lower:
        return 'adverse_effects_general'
    
    if 'cardiotoxic' in template_lower:
        return 'cardiotoxicity'
    elif 'nephrotoxic' in template_lower:
        return 'nephrotoxicity'
    elif 'hepatotoxic' in template_lower:
        return 'hepatotoxicity'
    elif 'neurotoxic' in template_lower:
        return 'neurotoxicity'
    elif 'myelosuppressive' in template_lower:
        return 'myelosuppression'
    elif 'ototoxic' in template_lower:
        return 'ototoxicity'
    
    if 'metabolism' in template_lower:
        return 'metabolism_change'
    elif 'serum concentration' in template_lower:
        if 'active metabolites' in template_lower:
            return 'metabolite_concentration_change'
        else:
            return 'serum_concentration_change'
    elif 'bioavailability' in template_lower:
        return 'bioavailability_change'
    elif 'absorption' in template_lower:
        return 'absorption_change'
    elif 'excretion rate' in template_lower:
        return 'excretion_change'
    elif 'protein binding' in template_lower:
        return 'protein_binding_change'
    
    if 'qtc' in template_lower or 'qt' in template_lower:
        return 'qtc_prolongation'
    elif 'bradycardic' in template_lower:
        return 'bradycardia'
    elif 'tachycardic' in template_lower:
        return 'tachycardia'
    elif 'arrhythmogenic' in template_lower:
        return 'arrhythmia'
    elif 'hypotensive' in template_lower and 'orthostatic' not in template_lower:
        return 'hypotension'
    elif 'hypertensive' in template_lower:
        return 'hypertension'
    elif 'orthostatic hypotensive' in template_lower:
        return 'orthostatic_hypotension'
    elif 'antihypertensive' in template_lower:
        return 'antihypertensive_effect'
    elif 'vasoconstricting' in template_lower or 'vasopressor' in template_lower:
        return 'vasoconstriction'
    elif 'vasodilatory' in template_lower:
        return 'vasodilation'
    elif 'heart failure' in template_lower:
        return 'heart_failure'
    elif 'av block' in template_lower or 'atrioventricular' in template_lower:
        return 'av_block'
    
    if 'anticoagulant' in template_lower:
        return 'anticoagulation'
    elif 'antiplatelet' in template_lower:
        return 'antiplatelet_effect'
    elif 'bleeding' in template_lower:
        return 'bleeding_risk'
    elif 'thrombogenic' in template_lower:
        return 'thrombosis'

    if 'hypokalemic' in template_lower:
        return 'hypokalemia'
    elif 'hyperkalemic' in template_lower or 'hyperkalemia' in template_lower:
        return 'hyperkalemia'
    elif 'hypocalcemic' in template_lower:
        return 'hypocalcemia'
    elif 'hypercalcemic' in template_lower:
        return 'hypercalcemia'
    elif 'hyponatremic' in template_lower:
        return 'hyponatremia'
    
    if 'cns depressant' in template_lower:
        if 'hypertensive' in template_lower:
            return 'cns_depression_and_hypertension'
        elif 'hypotensive' in template_lower:
            return 'cns_depression_and_hypotension'
        else:
            return 'cns_depression'
    elif 'neuroexcitatory' in template_lower:
        return 'neuroexcitation'
    elif 'sedative' in template_lower:
        return 'sedation'
    elif 'central neurotoxic' in template_lower:
        return 'central_neurotoxicity'
    elif 'serotonergic' in template_lower:
        return 'serotonergic_effect'
    elif 'antipsychotic' in template_lower:
        return 'antipsychotic_effect'
    
    if 'hypoglycemic' in template_lower:
        return 'hypoglycemia'
    elif 'hyperglycemic' in template_lower:
        return 'hyperglycemia'
    
    if 'respiratory depressant' in template_lower:
        return 'respiratory_depression'
    elif 'bronchodilatory' in template_lower:
        return 'bronchodilation'
    elif 'bronchoconstrictory' in template_lower:
        return 'bronchoconstriction'
    
    if 'neuromuscular blocking' in template_lower:
        return 'neuromuscular_blockade'
    elif 'adverse neuromuscular' in template_lower:
        return 'adverse_neuromuscular'
    elif 'myopathic rhabdomyolysis' in template_lower:
        return 'rhabdomyolysis'
    
    if 'therapeutic efficacy' in template_lower:
        return 'therapeutic_efficacy'
    elif 'analgesic' in template_lower:
        return 'analgesic_effect'
    elif 'anticholinergic' in template_lower:
        return 'anticholinergic_effect'
    elif 'immunosuppressive' in template_lower:
        return 'immunosuppression'
    elif 'diuretic' in template_lower:
        return 'diuretic_effect'
    elif 'stimulatory' in template_lower:
        return 'stimulation'
    
    if 'ulcerogenic' in template_lower:
        return 'ulcerogenic_effect'
    elif 'constipating' in template_lower:
        return 'constipation'

    if 'fluid retaining' in template_lower:
        return 'fluid_retention'
    
    if 'dermatologic' in template_lower:
        return 'dermatologic_adverse'
    
    if 'hypersensitivity' in template_lower:
        return 'hypersensitivity'
    
    if 'diagnostic agent' in template_lower:
        return 'diagnostic_interference'
    
    return 'other_interaction'

def extract_interaction_template(description, drug1, drug2):
    template = description
    template = re.sub(re.escape(drug1), 'DRUG_A', template, flags=re.IGNORECASE)
    template = re.sub(re.escape(drug2), 'DRUG_B', template, flags=re.IGNORECASE)
    template = re.sub(r'\bincreas(e|ed|es|ing)\b', 'DIRECTION', template, flags=re.IGNORECASE)
    template = re.sub(r'\bdecreas(e|ed|es|ing)\b', 'DIRECTION', template, flags=re.IGNORECASE)
    template = re.sub(r'\b(enhance|enhanced|enhances|enhancing)\b', 'DIRECTION', template, flags=re.IGNORECASE)
    template = re.sub(r'\b(reduc(e|ed|es|ing)|diminish(ed|es|ing)?|lower(ed|s|ing)?)\b', 'DIRECTION', template, flags=re.IGNORECASE)
    template = re.sub(r'\b(elevat(e|ed|es|ing)|rais(e|ed|es|ing))\b', 'DIRECTION', template, flags=re.IGNORECASE)
    return template

"""
df['Template'] = df.apply(
    lambda row: extract_interaction_template(
        row['Interaction Description'], 
        row['Drug 1'], 
        row['Drug 2']
    ), 
    axis=1
)
print(f"Created {df['Template'].nunique()} unique templates")
"""
"""
# Categorize the templates
df['Interaction_Category'] = df['Template'].apply(categorize_template)
print(f"\nCreated {df['Interaction_Category'].nunique()} base categories")

# Combine category + direction
df['Category_With_Direction'] = df['Interaction_Category'] + '_' + df['Direction']
print(f"Combined into {df['Category_With_Direction'].nunique()} final categories")
"""
"""
for i, label in enumerate(le.classes_):
    count = (df['Direction'] == label).sum()
    # Parse out category and direction for display
    if '_increased' in label:
        category = label.replace('_increased', '')
        direction = '↑'
    elif '_decreased' in label:
        category = label.replace('_decreased', '')
        direction = '↓'
    elif '_neutral' in label:
        category = label.replace('_neutral', '')
        direction = '='
    elif '_mixed' in label:
        category = label.replace('_mixed', '')
        direction = '±'
    else:
        category = label
        direction = '?'

    print(f"  {i:3d}. [{direction}] {category:.<45} {count:>6,} samples")
"""

In [47]:
def extract_direction(description):
    desc = description.lower()
    
    if re.search(r"\bincreas(e|ed)\b", desc):
        return "increased"
    
    if re.search(r"\bdecreas(e|ed)\b|\breduced\b", desc):
        return "decreased"
    
    return "other"

In [49]:
# add interaction label to dataset

df = pd.read_csv('drug_interactions_cleaned.csv')

df['Direction'] = df['Interaction Description'].apply(extract_direction)
print(df['Direction'].value_counts())

le = LabelEncoder()
df['Interaction_Label'] = le.fit_transform(df['Direction'])

num_classes = len(le.classes_)
print(f"Number of classes for training: {num_classes}")

df.to_csv('drug_interactions_direction.csv', index=False)

Direction
increased    128533
decreased     61399
Name: count, dtype: int64
Number of classes for training: 2


In [50]:
def smiles_to_fingerprint(smiles, radius=2, nBits=2048):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        fp = gen.GetFingerprint(mol)
        return np.array(fp)
    except:
        return None

In [52]:
df = pd.read_csv('drug_interactions_direction.csv')

unique_smiles = pd.concat([df['Drug1_SMILES'], df['Drug2_SMILES']]).unique()

smiles_to_fp_dict = {}
for smiles in tqdm(unique_smiles):
    fp = smiles_to_fingerprint(smiles)
    smiles_to_fp_dict[smiles] = fp

df['Drug1_FP'] = df['Drug1_SMILES'].map(smiles_to_fp_dict)
df['Drug2_FP'] = df['Drug2_SMILES'].map(smiles_to_fp_dict)

before_removal = len(df)
df = df.dropna(subset=['Drug1_FP', 'Drug2_FP'])
print(f"Removed: {before_removal - len(df)} rows with failed fingerprints")

 30%|██▉       | 496/1656 [00:00<00:01, 592.93it/s][17:05:41] WARNING: not removing hydrogen atom without neighbors
[17:05:41] WARNING: not removing hydrogen atom without neighbors
 61%|██████    | 1011/1656 [00:01<00:01, 634.72it/s][17:05:42] WARNING: not removing hydrogen atom without neighbors
[17:05:42] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 1656/1656 [00:02<00:00, 619.80it/s]

Removed: 0 rows with failed fingerprints


In [53]:
X1 = np.stack(df['Drug1_FP'].values).astype(np.float32)
X2 = np.stack(df['Drug2_FP'].values).astype(np.float32)
y = df['Interaction_Label'].values.astype(np.int64)

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    X1, X2, y, test_size=0.2, random_state=42
)

In [54]:
class DDIDataset(Dataset):
    def __init__(self, drug1_fps, drug2_fps, labels):
        self.drug1_fps = torch.FloatTensor(drug1_fps)
        self.drug2_fps = torch.FloatTensor(drug2_fps)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.drug1_fps[idx], self.drug2_fps[idx], self.labels[idx]

train_dataset = DDIDataset(X1_train, X2_train, y_train)
test_dataset = DDIDataset(X1_test, X2_test, y_test)

batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [55]:
class DDIPredictor(nn.Module):
    def __init__(self, num_classes):
        super(DDIPredictor, self).__init__()
        
        self.drug_encoder = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.MaxPool1d(2),
            
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.MaxPool1d(2),

            nn.Conv1d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.AdaptiveMaxPool1d(1)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 2, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, drug1, drug2):
        drug1 = drug1.unsqueeze(1)  # (batch, 1, 2048)
        drug2 = drug2.unsqueeze(1)  # (batch, 1, 2048)
    
        drug1_features = self.drug_encoder(drug1).squeeze(-1)  # (batch, 512)
        drug2_features = self.drug_encoder(drug2).squeeze(-1)  # (batch, 512)
        
        combined = torch.cat([drug1_features, drug2_features], dim=1)  # (batch, 1024)
        output = self.fc_layers(combined)

        return output

model = DDIPredictor(num_classes=num_classes).to(device)

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [57]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for drug1, drug2, labels in pbar:
        drug1, drug2, labels = drug1.to(device), drug2.to(device), labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(drug1, drug2)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * drug1.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({'loss': loss.item(), 'acc': correct/total})
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

In [58]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for drug1, drug2, labels in loader:
            drug1, drug2, labels = drug1.to(device), drug2.to(device), labels.to(device)
            
            outputs = model(drug1, drug2)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * drug1.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

In [59]:
num_epochs = 20
best_val_acc = 0.0

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rate': []
}

print("Starting training...\n")

for epoch in range(num_epochs):
    current_lr = optimizer.param_groups[0]['lr']
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['learning_rate'].append(current_lr)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] LR: {current_lr:.6f}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_cnn_model.pth')
        print(f"New best model saved (Val Acc: {val_acc:.4f})")
    print()

print("\nTraining complete!")
print(f"Best validation accuracy: {best_val_acc:.4f}")

Starting training...



Training:   1%|          | 13/2375 [01:04<3:14:20,  4.94s/it, loss=0.717, acc=0.53] 


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history['train_acc'], label='Train Accuracy')
axes[0].plot(history['val_acc'], label='Val Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss plot
axes[1].plot(history['train_loss'], label='Train Loss')
axes[1].plot(history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

best_epoch = np.argmax(history['val_acc'])
print(f"Best epoch: {best_epoch + 1}")
print(f"Best validation accuracy: {history['val_acc'][best_epoch]:.4f}")

In [ ]:
# Evaluate on test set with best model
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print(f"{'='*60}")
print(f"Final Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Final Test Loss: {test_loss:.4f}")
print(f"{'='*60}")

In [ ]:
def smiles_to_fingerprint(smiles, radius=2, nBits=2048):
    """Convert SMILES to Morgan fingerprint"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        fp = gen.GetFingerprint(mol)
        return np.array(fp, dtype=np.float32)
    except:
        return None

def predict_interaction(drug1_smiles, drug2_smiles, top_k=3):
    """Predict interaction between two drugs"""
    model.eval()
    
    # Convert to fingerprints
    fp1 = smiles_to_fingerprint(drug1_smiles)
    fp2 = smiles_to_fingerprint(drug2_smiles)
    
    if fp1 is None or fp2 is None:
        return [("Invalid SMILES", 0.0)]
    
    # Convert to tensors
    fp1 = torch.FloatTensor(fp1).unsqueeze(0).to(device)
    fp2 = torch.FloatTensor(fp2).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(fp1, fp2)
        probabilities = torch.softmax(outputs, dim=1)[0]
    
    # Get top k predictions
    top_probs, top_indices = torch.topk(probabilities, top_k)
    
    results = []
    for prob, idx in zip(top_probs, top_indices):
        interaction_type = le.inverse_transform([idx.item()])[0]
        results.append((interaction_type, prob.item()))
    
    return results

print("✓ Prediction function defined")

In [ ]:
# Test predictions
print("Testing predictions on random samples:\n")
print("="*80)

test_samples = df.sample(5, random_state=42)

for idx, row in test_samples.iterrows():
    print(f"\n{'='*80}")
    print(f"Drug 1: {row['Drug 1']}")
    print(f"Drug 2: {row['Drug 2']}")
    print(f"\nActual Interaction:")
    print(f"  {row['Interaction Description']}")
    
    predictions = predict_interaction(row['Drug1_SMILES'], row['Drug2_SMILES'], top_k=3)
    
    print(f"\nTop 3 Predictions:")
    for i, (interaction, conf) in enumerate(predictions, 1):
        match = "✓" if interaction == row['Interaction Description'] else "✗"
        print(f"  {i}. [{match}] {interaction}")
        print(f"      Confidence: {conf:.2%}")

print(f"\n{'='*80}")